In [ ]:
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/data/hand_landmark_logging.csv", encoding="cp949")
data.dropna(inplace=True)
data

In [185]:
# max_frame 이상 또는 min_frame 미만인 label과 count로 구성된 list를 반환
def get_drop_index_list(df, min_frame, max_frame):
  drop_label1 = pd.DataFrame(df[['count', 'label']].value_counts() > max_frame)
  drop_label2 = pd.DataFrame(df[['count', 'label']].value_counts() < min_frame)
  drop_label = pd.concat([drop_label1, drop_label2])
  drop_label.reset_index(inplace=True)
  drop_label_df = drop_label[drop_label[0] == True]
  drop_index_list = []
  for idx, row in drop_label_df.iterrows():
    drop_index_list.append([row['label'], row['count']])
  return drop_index_list

# 특정 라벨에 해당하는 동작의 프레임 전체 반환
def label_dataframe(label_idx, df):
  result_df = pd.DataFrame(columns = df.columns)
  for idx, row in df.iterrows():
    if row['label'] == label_idx:
      result_df.loc[len(result_df)] = row
  return result_df

In [186]:
# 특정 라벨값에 대하여 프레임 수가 70 초과인 행의 인덱스 반환
def drop_row(label_idx_list, df):
  result_df = pd.DataFrame(columns=df.columns)
  for label_idx in label_idx_list:
    result_df = pd.concat([result_df, label_dataframe(label_idx, df)], ignore_index=True)
    drop_index_list = get_drop_index_list(df, 40, 70)
    for i in range(len(drop_index_list)):
      if drop_index_list[i][0] == label_idx:
        drop_idx = result_df[(result_df['label'] == label_idx) & (result_df['count'] == drop_index_list[i][1])].index
        result_df.drop(drop_idx, inplace=True)
  return result_df

In [187]:
# 손 펴고 있는 동작 로그 (count, label 컬럼 포함X)
hands_default = {"wrist_x": [0], "wrist_y": [0], "thumb_cmc_x": -0.175258, "thumb_cmc_y": -0.087629, "thumb_mcp_x": -0.329897, "thumb_mcp_y": -0.206186, "thumb_ip_x": -0.469072, "thumb_ip_y": -0.273196,
                     "thumb_tip_x": -0.592784, "thumb_tip_y": -0.278351, "index_finger_mcp_x": -0.149485, "index_finger_mcp_y": -0.489691, "index_finger_pip_x": -0.195876, "index_finger_pip_y": -0.701031,
                     "index_finger_dip_x": -0.221649, "index_finger_dip_y": -0.829897, "index_finger_tip_x": -0.231959, "index_finger_tip_y": -0.938144, "middle_finger_mcp_x": -0.025773, "middle_finger_mcp_y": -0.515464,
                     "middle_finger_pip_x": -0.025773, "middle_finger_pip_y": -0.742268, "middle_finger_dip_x": -0.025773, "middle_finger_dip_y": -0.881443, "middle_finger_tip_x": -0.025773, "middle_finger_tip_y": -1.0,
                     "ring_finger_mcp_x" : 0.092784, "ring_finger_mcp_y": -0.489691, "ring_finger_pip_x": 0.118557, "ring_finger_pip_y": -0.695876, "ring_finger_dip_x": 0.134021, "ring_finger_dip_y": -0.829897,
                     "ring_finger_tip_x": 0.144330, "ring_finger_tip_y": -0.938144, "pinky_mcp_x": 0.201031, "pinky_mcp_y": -0.422680, "pinky_pip_x": 0.283505, "pinky_pip_y": -0.551546, "pinky_dip_x": 0.340206,
                     "pinky_dip_y": -0.639175, "pinky_tip_x": 0.381443, "pinky_tip_y": -0.721649}
default_hand = pd.DataFrame(hands_default)

In [188]:
# 각 count에 대한 프레임 수를 데이터프레임으로 반환
def get_frame_count(df):
  frame_count = pd.DataFrame(first_preprocessing_df.groupby('label')['count'].value_counts())
  frame_count.rename(columns={'count': 'frame_count'}, inplace=True)
  frame_count.reset_index(inplace=True)
  return frame_count.drop(frame_count[frame_count['frame_count'] == 56].index)

In [189]:
def get_frames(df, count_df, frame_num):
  remain = []
  for idx, row in count_df.iterrows():
    remain.append([row['label'], row['count'], row['frame_count']])
  insert_df = pd.DataFrame(columns=df.columns)
  for i in range(len(remain)):
    if remain[i][2] == frame_num:
      continue;
    # 프레임수 부족
    elif remain[i][2] < frame_num:
      for label_idx in label_idx_list:
        default_hand = pd.DataFrame(hands_default)
        default_hands = pd.DataFrame(columns=df.columns)
        default_hand.insert(0, 'label', remain[i][0])
        default_hand.insert(0, 'count', remain[i][1])
        drop_frame = df[(df['count'] == remain[i][1]) & (df['label'] == remain[i][0])]
        if drop_frame.empty == False:
          for w in range(0, frame_num - int(remain[i][2])):
              default_hands = pd.concat([default_hands, default_hand], ignore_index=True)
        else:
          break;
        insert_df = pd.concat([insert_df, default_hands, drop_frame],ignore_index=True)
        df.drop(drop_frame.index, axis=0, inplace=True)

    # 프레임수 초과
    elif remain[i][2] > frame_num:
      half = (int(remain[i][2]) - frame_num) // 2
      for _ in range(0, half):
        df.drop(df[(df['count'] == remain[i][1]) & (df['label'] == remain[i][0])].index[0], axis=0, inplace=True)
      for _ in range(0, int(remain[i][2]) - (frame_num + half)):
        df.drop(df[(df['count'] == remain[i][1]) & (df['label'] == remain[i][0])].index[-1], axis=0, inplace=True)
  result = pd.concat([insert_df, df], ignore_index=True)

  return result

In [190]:
label_idx_list = [0,1,2,3,4,5,6,7,8,9,10,11]
first_preprocessing_df = drop_row(label_idx_list, data)

frame_count = get_frame_count(first_preprocessing_df)

preprocessed_data = get_frames(first_preprocessing_df, frame_count, 56)
preprocessed_data.to_csv("preprocessed_data.csv")

In [ ]:
pd.set_option('display.max_rows', None)
preprocessed_data.groupby('label')['count'].value_counts()